### **FUENTES**:

PetFinder Kaggle:

https://www.kaggle.com/competitions/petfinder-adoption-prediction/data

First Tutorial:

https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5

Second Deep Tutorial:

https://rumn.medium.com/part-1-ultimate-guide-to-fine-tuning-in-pytorch-pre-trained-model-and-its-configuration-8990194b71e

Logo Recognition API:

https://heartbeat.comet.ml/logo-recognition-ios-application-using-machine-learning-and-flask-api-aec4eff3be11

Hybrid (multimodal) neural network architecture : Combination of tabular, textual and image inputs:

https://medium.com/@dave.cote.msc/hybrid-multimodal-neural-network-architecture-combination-of-tabular-textual-and-image-inputs-7460a4f82a2e



### **INDICACIONES PREVIAS**:

+ **Git**:
    + Clonamos el repo: root de todos los repos y ponemos git clone "url_repo"
    + Hacemos el checkout de la rama main: git checkout -b new-branch

+ **Poetry**:
    + Instalamos poetry: https://python-poetry.org/docs/
    + Realizamos un Update del pyproject: poetry update
    + Activamos el entorno que creo poetry: poetry shell
    + Intentamos correr una celda, si nos pide seleccionar el environment y no lo vemos en la lista, cerrar y volver abrir VSC

+ **Torch y CUDA**:
    + Verificar que versión pide torch:
        + Versión de torch instalada: poetry show (en mi caso la 1.13.1)
        + Buscar la versión correspondiente en la documentación: https://pytorch.org/get-started/previous-versions/  (en mi caso el 11.7)
    + Instalar CUDA para Torch (buscar la versión correspondiente de CUDA): https://developer.nvidia.com/cuda-11-7-0-download-archive
    + Verificar que CUDA esté funcional: correr en una celda torch.cuda.is_available()

In [1]:
#!pip install augment

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import os
import sys
import shutil
import time
import copy
import datetime
from tqdm import tqdm
#import matplotlib.pyplot as plt
#import seaborn as sns
#import cv2
#from PIL import Image
#from pathlib import Path

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F

from joblib import load, dump

from utils import plot_confusion_matrix

# Appendeo el directorio de una carpeta arriba
nb_dir = os.path.dirname(os.path.abspath("05_Resnet50_3_train_augment.ipynb"))
dir = os.path.abspath(os.path.join(nb_dir,'..'))
sys.path.append(dir)

# Set the path to the augment folder
path_to_augment = r'c:\Users\Miner\MCD\LaboII\augment_lib'

# Add the folder to sys.path
sys.path.append(os.path.abspath(path_to_augment))

# Now import the autoaugment package
from autoaugment import ImageNetPolicy
from cutout import Cutout

# Verificamos que CUDA está funcional
torch.cuda.is_available()

c:\anaconda3\envs\ldi2_cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# Print the current working directory
print(os.getcwd())

c:\Users\Miner\MCD\LaboII\tutoriales


In [4]:
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.version.cuda)         # PyTorch CUDA version

True
1
12.1


**Seteo el Modelo**

Teoría de Resnet: https://towardsdatascience.com/introduction-to-resnets-c0a830a288a4

In [5]:
# Importo modelo ResNet entrenado en Imagenet
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# Modificar la última capa para adaptarse a tu problema específico
num_ftrs = resnet50.fc.in_features
resnet50.fc = torch.nn.Linear(num_ftrs, 5) # Clasificación 5 clases
# Configuro para usar cuda si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
# Instancio del criterio de pérdida CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
# Instancio Stochastic Gradient Descent (SGD): Defino el parámetro del Learning Rate (define "el paso" en que avanzan los pesos en cada iteración) y el Momentum (pone innercia a la dirección del gradiente descendiente para que no cambie de dirección en minimos locales)
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9) # Parámetros default del SGD


**Seteo parámetros, directorios y funciones**

In [6]:
# Paths
BASE_DIR = '../'
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")
PATH_TO_IMAGES_DIR = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train_images")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")

MODEL_NAME = '04 ResNet'

MODEL_VERSION = '5.0.1'

# Parametros y variables
CREATE_PYTORCH_DIRECTORIES = 1
SEED = 42
BATCH_SIZE = 4
TEST_SIZE = 0.2
IMAGE_SIZE = 299
CPU_CORES = os.cpu_count()

# Armo el nuevo directorio de train
new_train_directory = os.path.join(BASE_DIR, 'work/train_images_classes')
os.makedirs(new_train_directory, exist_ok=True) # si ya existe el nombre, lo deja como está

# Armo el nuevo directorio de validación
new_val_directory = os.path.join(BASE_DIR, 'work/val_images_classes')
os.makedirs(new_val_directory, exist_ok=True)

# Definir las clases ordenadas
class_names = ['0', '1', '2', '3', '4']

# Mapear las etiquetas de las clases a números enteros consecutivos
class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

# Creo las carpetas de clases dentro de los directorios
for clase in class_names: # Una para cada clase
   os.makedirs(os.path.join(new_train_directory, str(clase)), exist_ok=True)
   os.makedirs(os.path.join(new_val_directory, str(clase)), exist_ok=True)




# Funciones para la carga y el preproceso
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    # Calcula el factor de escala necesario para redimensionar la imagen de manera que el lado más largo tenga el tamaño deseado 
    ratio = float(IMAGE_SIZE)/max(old_size)
    # Calcula las nuevas dimensiones de la imagen 
    new_size = tuple([int(x*ratio) for x in old_size])
    # Redimensiona la imagen con el nuevo tamaño
    im = cv2.resize(im, (new_size[1], new_size[0]))
    # Calcula las diferencias de tamaño y agrega pixeles (color negro) en los extremos para que quede centrada y cuadrada 
    delta_w = IMAGE_SIZE - new_size[1]
    delta_h = IMAGE_SIZE - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_image = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_image


def load_image(pet_id):
    path_to_image = os.path.join(PATH_TO_IMAGES_DIR, f'{pet_id}-1.jpg') # Irá a la primera imagen de la mascota
    image = cv2.imread(path_to_image)
    # Convierte la imagen de BGR a RGB porque estos modelos esperan ese orden de canales
    image = cv2.convertScaleAbs(image)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    new_image = resize_to_square(image)
    return new_image


In [7]:

def visualize_pet(pet_id):
    path_to_image = os.path.join(PATH_TO_IMAGES_DIR, f'{pet_id}-1.jpg') # Irá a la primera imagen de la mascota
    # Cargar la imagen
    image_to_show = cv2.imread(path_to_image)
    # Convertir a formato RGB
    image_to_show = cv2.cvtColor(image_to_show, cv2.COLOR_BGR2RGB)
    # Visualizar la imagen
    plt.imshow(image_to_show)
    plt.axis('off')  # No mostrar los ejes
    plt.show()

def visualize_image(image):
    # Convierte la imagen a un formato de enteros (CV_8U)
    image = cv2.convertScaleAbs(image)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Visualizar la imagen
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')  # No mostrar los ejes
    plt.show()


**Cargo y Proceso Data**

Nota: Pytorch necesita que estén las imágenes en los distintos directorios según su clase y su participación en el training

In [8]:
# Cargo
train_df = pd.read_csv(PATH_TO_TRAIN)

# Split para validación
train_data, val_data = train_test_split(train_df,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = train_df.AdoptionSpeed)




if CREATE_PYTORCH_DIRECTORIES == 1: # Poner en 0 si ya tengo las carpetas train_images_classes y val_images_classes con las imágenes copiadas
    # Función para copiar las imágenes a los directorios correspondientes
    def copy_imag(data, directorio_destino):
        for index, row in data.iterrows():
            petID = row['PetID']
            adoption_speed = row['AdoptionSpeed']
            
            # Nombre del archivo de imagen
            nombre_archivo = f"{petID}-1.jpg"
            
            # Ruta completa de la imagen de origen
            ruta_origen = os.path.join(PATH_TO_IMAGES_DIR, nombre_archivo)
            
            # Ruta completa del directorio de destino
            ruta_destino = os.path.join(directorio_destino, str(adoption_speed), nombre_archivo)
            
            # Verificar si el archivo de origen existe
            if os.path.exists(ruta_origen):
                # Copiar el archivo de origen al directorio de destino
                shutil.copy2(ruta_origen, ruta_destino)
        print("Completada la copia a: ",str(directorio_destino))

    # Copiar las imágenes al directorio de train
    copy_imag(train_data, new_train_directory)

    # Copiar las imágenes al directorio de val
    copy_imag(val_data, new_val_directory)

    print("Proceso completado.")

Completada la copia a:  ../work/train_images_classes
Completada la copia a:  ../work/val_images_classes
Proceso completado.


In [9]:
# Genero los DataLoaders
def create_dataloaders(train_directory, val_directory, batch_size, num_workers):
    # Transformaciones de imagen para el conjunto de entrenamiento
    train_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),
        ImageNetPolicy(),  ############### LAS POLÍTICAS AUTOAUGMENT PARA IMAGENET (AUGMENT)
        transforms.ToTensor(),
        Cutout(n_holes=1, length = 16),   ############### CUTOUT PARA REGULARIZACIÓN (AUGMENT)
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Transformaciones de imagen para el conjunto de validación (sin data augment)
    val_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Crear conjuntos de datos para el conjunto de entrenamiento y validación
    conjunto_entrenamiento = datasets.ImageFolder(train_directory, transform=train_transforms)
    conjunto_validacion = datasets.ImageFolder(val_directory, transform=val_transforms)

    # Asignar las clases ordenadas al conjunto de datos
    conjunto_entrenamiento.class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}
    conjunto_validacion.class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

    # Crear dataloaders para el conjunto de entrenamiento y validación
    train_dataloader = torch.utils.data.DataLoader(conjunto_entrenamiento, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_dataloader = torch.utils.data.DataLoader(conjunto_validacion, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_dataloader, val_dataloader

# Aplico las funcion de los DataLoaders
train_dataloader, val_dataloader = create_dataloaders(new_train_directory , new_val_directory , BATCH_SIZE, CPU_CORES)

In [10]:
#Genero una lista de PetIDs con imagen en el orden en que aparecen en el data loader
test_sample_ids = [i[0].split('\\')[-1].split('-')[0] for i in val_dataloader.dataset.samples]

In [11]:
test_sample_ids

['015da9e87',
 '022606901',
 '02f89bdcb',
 '0cf7fae9d',
 '0e922caab',
 '1347325db',
 '142da3f69',
 '1557f91e8',
 '164857bd8',
 '1c60a65e8',
 '1eb1f56ed',
 '20bad295b',
 '23e49bd2e',
 '24e0380f1',
 '259b10a3e',
 '2f7563006',
 '3316a5619',
 '36e1415c8',
 '383e5b82e',
 '390689a38',
 '3bbbccce4',
 '416cb5d6d',
 '484346fbc',
 '497d68d48',
 '4aa9a342e',
 '4d163b731',
 '5376c81df',
 '54e731fab',
 '596cf7334',
 '5a089c77f',
 '5ca64db1a',
 '5d808057a',
 '5dd7c4c06',
 '615213076',
 '61afb7281',
 '651d63dd7',
 '6607c6e22',
 '66b206dbe',
 '699a81c51',
 '6a72cfda7',
 '6b7be833c',
 '6c61c33ee',
 '6db27dbb7',
 '71deb8267',
 '77f9b1470',
 '79bf5eb37',
 '85078bc71',
 '8693c4e7f',
 '8b8c686ec',
 '95ee4775c',
 '96b7978d2',
 '999d24492',
 '9be11ef9b',
 '9c7917558',
 '9d4cf29f6',
 '9f3fe015b',
 '9ff766a26',
 'a0825ad18',
 'a0c15424b',
 'a12589a09',
 'a46ad5cc3',
 'a4edd47eb',
 'acde4eb5b',
 'ad2b462ee',
 'ae946bb63',
 'b5f846518',
 'bf458a7d4',
 'ceee65059',
 'd76c4ec1e',
 'd83aa1f02',
 'd9b63ae80',
 'e42d

**Entreno**

In [12]:
def train_val(model, criterion, optimizer, dataloaders, datasets, device, num_epochs=20, lr=0.001, momentum = 0.9 ,trial=None):
    
    # Instancio Stochastic Gradient Descent (SGD): Defino el parámetro del Learning Rate (define "el paso" en que avanzan los pesos en cada iteración) y el Momentum (pone innercia a la dirección del gradiente descendiente para que no cambie de dirección en minimos locales)
    optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=momentum) # Parámetros default del SGD
    
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_kappa =  -999

    train_losses = []
    val_losses = []

    try:
        previous_best = study.best_value
    except:
        previous_best = -999


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        kappa_labels_true = []
        kappa_labels_predicted = []
        output_scores = []

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)



                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    elif phase == 'val':
                        kappa_labels_true.extend(labels.cpu().numpy().tolist())
                        kappa_labels_predicted.extend(preds.cpu().numpy().tolist())
                        outputs_np = outputs.cpu().numpy()
                        output_scores.extend([outputs_np[i,:] for i in range(outputs_np.shape[0])])

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                #END OF BATCH

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects.double() / len(datasets[phase])
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                kappa_score = np.nan
            else:
                val_losses.append(epoch_loss)
                kappa_score = cohen_kappa_score(kappa_labels_true,
                                  kappa_labels_predicted,
                                  weights = 'quadratic')
                    


            print(f'{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc*100:.2f}% Kappa: {kappa_score:.3f}')

            # If this is the best Epoch so far -> Deep copy the model
            if phase == 'val' and kappa_score > best_kappa:
                best_acc = epoch_acc
                best_kappa = kappa_score
                best_model_wts = copy.deepcopy(model.state_dict())


                #Best Epoch within a trial and better than previous trials
                if trial is not None and best_kappa > previous_best:

                    #Save test dataset with predictions
                    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
                    predicted_df = pd.DataFrame({'PetID':test_sample_ids,
                                'pred':output_scores}).merge(val_data, on='PetID')
                    #display(predicted_df.head())
                    dump(predicted_df, predicted_filename)

                    #Generate and save CM 
                    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
                    plot_confusion_matrix(kappa_labels_true,kappa_labels_predicted).write_image(cm_filename)

            #END OF PHASE

        #END OF EPOCH

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.2f}%'.format(best_acc * 100))

    # Load best model weights
    model.load_state_dict(best_model_wts)

    # Save in optuna trial the best test dataset, cm and model weights
    if trial is not None and best_kappa > previous_best:
        upload_artifact(trial, predicted_filename, artifact_store)   

        upload_artifact(trial, cm_filename, artifact_store)

        file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{trial.number}.pth'
        model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
        torch.save(model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
        upload_artifact(trial, model_path, artifact_store)

    return model,best_kappa

best_model,_ = train_val(resnet50, criterion, optimizer,
                       dataloaders={'train': train_dataloader, 
                                    'val': val_dataloader}, 
                       datasets={'train': train_data, 'val': val_data}, 
                       device=device, 
                       num_epochs=4)    # 4 epochs
# Guardo el modelo
run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{run_id}.pth'
model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
torch.save(best_model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
print(f'Modelo guardado en {model_path}')

Epoch 0/3
----------


100%|██████████| 2931/2931 [04:25<00:00, 11.06it/s]


Train Loss: 1.3973 Acc: 32.54% Kappa: nan


100%|██████████| 733/733 [00:48<00:00, 15.03it/s]


Val Loss: 1.3740 Acc: 34.28% Kappa: 0.300
Epoch 1/3
----------


...

Val Loss: 1.4074 Acc: 33.84% Kappa: 0.246
Epoch 3/3
----------


100%|██████████| 2931/2931 [04:07<00:00, 11.86it/s]


Train Loss: 1.3190 Acc: 39.10% Kappa: nan


100%|██████████| 733/733 [00:46<00:00, 15.75it/s]

Val Loss: 1.3832 Acc: 34.28% Kappa: 0.284
Training complete in 20m 7s
Best val Acc: 34.28%
Modelo guardado en ../work/optuna_temp_artifacts\04 ResNet_5.0.0_20240923_161821.pth


In [13]:
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)


def optuna_train(trial):

    epochs = trial.suggest_int('epochs', 5, 5)

    lr = trial.suggest_float('lr', 0.00001, 0.1, log=True)

    momentum = trial.suggest_float('momentum', 0.0, 0.95)

    _,best_score = train_val(resnet50, criterion, optimizer,
                       dataloaders={'train': train_dataloader, 
                                    'val': val_dataloader}, 
                       datasets={'train': train_data, 'val': val_data}, 
                       device=device, 
                       num_epochs=epochs,
                       lr=lr,
                       momentum = momentum,
                       trial=trial)


    return(best_score)

C:\Users\Miner\AppData\Local\Temp\ipykernel_12948\3870832202.py:1: ExperimentalWarning: FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.
  artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)


In [14]:
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name=f'{MODEL_NAME}_{MODEL_VERSION}',
                            load_if_exists = True)
study.optimize(optuna_train, n_trials=30)    # 30 trials

[I 2024-09-23 16:18:24,179] A new study created in RDB with name: 04 ResNet_5.0.0


Epoch 0/4
----------


100%|██████████| 2931/2931 [04:05<00:00, 11.95it/s]


Train Loss: 1.3537 Acc: 36.34% Kappa: nan


100%|██████████| 733/733 [00:46<00:00, 15.92it/s]


...

100%|██████████| 2931/2931 [04:06<00:00, 11.90it/s]


Train Loss: 1.2980 Acc: 40.25% Kappa: nan


100%|██████████| 733/733 [00:46<00:00, 15.76it/s]
C:\Users\Miner\AppData\Local\Temp\ipykernel_12948\1984787669.py:123: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

C:\Users\Miner\AppData\Local\Temp\ipykernel_12948\1984787669.py:125: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

C:\Users\Miner\AppData\Local\Temp\ipykernel_12948\1984787669.py:130: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.



Val Loss: 1.3797 Acc: 34.54% Kappa: 0.284
Training complete in 24m 22s
Best val Acc: 35.28%


[I 2024-09-23 17:07:10,645] Trial 1 finished with value: 0.3022574162937032 and parameters: {'epochs': 5, 'lr': 0.0008733451027144203, 'momentum': 0.1707633893163764}. Best is trial 1 with value: 0.3022574162937032.
